In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import soundfile as sf
from pathlib import Path
import re
import json
from pathlib import Path

This Notebook producess annotation for the audio MNIST dataset - spoken digit recognition 

https://github.com/Jakobovski/free-spoken-digit-dataset

Dataset it partitioned into the different digits and into the different speakers reading out the digits

The data and annotations can also be found in our public s3 bucket:
s3://deepvoice-external/free-spoken-digit-dataset

# Create annotation

In [6]:
file_path = 'free-spoken-digit-dataset/data'

In [7]:
file_path_obj = Path(file_path)

In [8]:
all_files = list(file_path_obj.glob('*.wav'))

In [10]:
n_files = len(all_files)

In [33]:
annotation_df = pd.DataFrame(data=np.concatenate([np.ones((n_files, 1))*0, np.ones((n_files, 1))*0.14, 
                                                  np.array([all_files[i].stem for i in range(n_files)]).reshape((-1,1)),
                                                  np.ones((n_files, 1))*0.14,
                                                  np.zeros((n_files, 1))],axis=1),
                             columns=['begin_time', 'end_time', 'filename', 'call_length', 'label'])
annotation_df.head(2)

,begin_time,end_time,filename,call_length,label
0,0.0,0.14,5_nicolas_12,0.14,0.0
1,0.0,0.14,6_yweweler_11,0.14,0.0


## remove all but 0 and 5 to make it a 2 class problem

In [36]:
annotation_df = annotation_df[annotation_df.filename.str[0].isin(['0', '5'])]

In [40]:
annotation_df['label'] = annotation_df.filename.str[0].isin(['0'])*1

<ipython-input-40-47be75d13b25>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_df['label'] = annotation_df.filename.str[0].isin(['0'])*1


In [42]:
annotation_df.head(3)

,begin_time,end_time,filename,call_length,label
0,0.0,0.14,5_nicolas_12,0.14,0
2,0.0,0.14,0_jackson_31,0.14,1
3,0.0,0.14,0_jackson_0,0.14,1


## split train test, using all dtata spoken by nicolas as the test set

In [44]:
annotation_df[annotation_df.filename.str.contains('nicolas')].to_csv('digit_annotation_test.csv', index=False)

In [45]:
annotation_df[~annotation_df.filename.str.contains('nicolas')].to_csv('digit_annotation_train.csv', index=False)